In [1]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

(<keras.engine.topology.InputLayer object at 0x7f68345be750>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f683b0d95d0>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e3938990>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f67e3938b10>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e3892e90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e38f6fd0>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f67e38ba690>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e384ac90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e384ad10>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e3862f10>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f67e3807d50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e3830e50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e35c0cd0>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f67e35d6ad0>, False)
(<keras.layers.poolin

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(19, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 19)                9747      
Total params: 40,940,371
Trainable params: 33,305,107
Non-trainable params: 7,635,264
________________________________________________________

In [5]:
train_dir = 'crowdai_train/'
validation_dir = 'crowdai_test/'

In [6]:
import numpy as np
from random import shuffle

In [7]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 62
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=True)

Found 8280 images belonging to 19 classes.
Found 2017 images belonging to 19 classes.


In [8]:
print train_generator.class_indices

{'Tomato Early Blight': 7, 'Corn Gray Leaf Spot': 3, 'Tomato LeafMold': 9, 'grape black rot': 16, 'TomatoBacterial Spot ': 12, 'Tomato Late Blight': 8, 'Potato Healthy': 6, ' Potato Late Blight ': 1, 'grape leaf blight': 18, 'TomatoTwo Spotted Spider Mite': 13, ' Corn Northern Leaf Blight': 0, 'Tomato Yellow Leaf Curl': 11, 'Corn CommonRust': 2, 'Potato Early Blight': 5, 'Corn healthy': 4, 'grape healthy': 17, 'Tomatohealthy': 14, 'Tomato Target Spot': 10, 'grape black measles': 15}


In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Epoch 1/30
133/133 [==============================] - 47s 354ms/step - loss: 0.9738 - acc: 0.6920 - val_loss: 0.3840 - val_acc: 0.8891
Epoch 2/30
133/133 [==============================] - 46s 347ms/step - loss: 0.3298 - acc: 0.8895 - val_loss: 0.2623 - val_acc: 0.9123
Epoch 3/30
133/133 [==============================] - 46s 347ms/step - loss: 0.1898 - acc: 0.9380 - val_loss: 0.3428 - val_acc: 0.8804
Epoch 4/30
133/133 [==============================] - 45s 341ms/step - loss: 0.1199 - acc: 0.9644 - val_loss: 0.1741 - val_acc: 0.9407
Epoch 5/30
133/133 [==============================] - 46s 344ms/step - loss: 0.0761 - acc: 0.9762 - val_loss: 0.2041 - val_acc: 0.9492
Epoch 6/30
133/133 [==============================] - 44s 331ms/step - loss: 0.0685 - acc: 0.9798 - val_loss: 0.3104 - val_acc: 0.9238
Epoch 7/30
133/133 [==============================] - 46s 347ms/step - loss: 0.0618 - acc: 0.9840 - val_loss: 0.2484 - val_acc: 0.9238
Epoch 8/30
133/133 [==============================] - 4

KeyboardInterrupt: 

In [10]:
import cv2

In [27]:
img3 = cv2.imread('/combo/Shashank/Tom_hl.JPG')
img3 = cv2.resize(img3,(224,224))
img3 = np.reshape(img3,[1,224,224,3])

In [28]:
disease = model.predict_classes(img3)
print disease

[14]


In [29]:
model.save('plant_disease_final.h5')